In [1]:
import pandas as pd
from cleantext import clean
import re
import torch.nn.functional as F


import torch
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
from pandas import read_excel, read_json
import sys
import time
import math

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
/home/remi/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
window_size = 310
slide = 70

In [3]:
def make_windows(content, w_size, slide):
    if slide > w_size :
        raise Exception()
    tokens = content.split(" ")
    l = len(tokens)    
       
    n = math.ceil(l / (w_size-slide))        

    if n==1 :
        return [content]
    else :
        windows = []
        for i in range(n) :            
            x = i*slide
            if x < l :
                y = x + w_size
                w = tokens[x:y]
                windows.append(" ".join(w))
        
        return windows

In [4]:
fiches = pd.read_csv("/home/remi/Downloads/xp-documents-fiches (2).csv")

fiches["cleaned_text"] = fiches["text"].apply(lambda t : clean(re.sub("\s+", " ", str(t)), to_ascii=False))
fiches["tokens"] = fiches["cleaned_text"].apply(lambda t : len(str(t).split(" ")))

fiches = fiches[fiches["tokens"]<3000]
fiches["windows"] = fiches["cleaned_text"].apply(lambda t : make_windows(t, window_size, slide))
fiches.drop(columns=["cleaned_text", "text"], inplace=True)
fiches = fiches.explode("windows", ignore_index=True)
fiches.rename(columns={"windows": "text"}, inplace=True)
fiches["cdtnId-fiches"] = fiches["cdtnId"]
fiches["cdtnId"] = fiches["cdtnId"].apply(lambda x : x.split("-")[0])

In [5]:
contribs = pd.read_csv("/home/remi/Downloads/xp-documents-contribs (1).csv")

In [6]:
contribs.drop_duplicates(subset=["answers.generic.text"], inplace=True)
contribs["cleaned_text"] = contribs["answers.generic.text"].apply(lambda t : clean(re.sub("\s+", " ", str(t)), to_ascii=False))
contribs["tokens"] = contribs["cleaned_text"].apply(lambda t : len(str(t).split(" ")))
contribs["windows"] = contribs["cleaned_text"].apply(lambda t : make_windows(t, window_size, slide))
contribs.drop(columns=["cleaned_text","answers.generic.text","text"], inplace=True)
contribs = contribs.explode("windows", ignore_index=True)
contribs.rename(columns={"windows": "text"}, inplace=True)

In [7]:
contribs["slug"] = contribs["slug"].apply(lambda s: "-".join(s.split("-")[1:]))

In [8]:
contribs_tensor = torch.load('/home/remi/Desktop/contribs_tensor3.pt')
fiches_tensor = torch.load('/home/remi/Desktop/fiches_tensor3.pt')
tensor_docs = torch.vstack([fiches_tensor, contribs_tensor])

In [9]:
passages_set = pd.concat([fiches, contribs]).reset_index()
passages_set.shape

(4795, 9)

In [10]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-base')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-base')

In [11]:
# fiche-service-public/un-salarie-peut-il-reporter-des-jours-de-conges-payes-pour-cause-de-maladie	
doc1 = "6e888b3f6e"

# contribution/si-le-salarie-est-malade-pendant-ses-conges-quelles-en-sont-les-consequences
doc2 = "1100b865de"

passages_set[passages_set["cdtnId"] == doc1]

,index,source,cdtnId,title,slug,answers.generic.text,tokens,text,cdtnId-fiches
3568,3568,fiches_service_public,6e888b3f6e,Arrêt maladie et congés payés annuels,arret-maladie-et-conges-payes-annuels,-,741,que se passe-t-il si vous tombez malade pendan...,6e888b3f6e
3569,3569,fiches_service_public,6e888b3f6e,Arrêt maladie et congés payés annuels,arret-maladie-et-conges-payes-annuels,-,741,de congés restants si la convention collective...,6e888b3f6e
3570,3570,fiches_service_public,6e888b3f6e,Arrêt maladie et congés payés annuels,arret-maladie-et-conges-payes-annuels,-,741,congés si vous êtes malade (maladie non-profes...,6e888b3f6e
3571,3571,fiches_service_public,6e888b3f6e,Arrêt maladie et congés payés annuels,arret-maladie-et-conges-payes-annuels,-,741,prise de congés en cours dans l'entreprise ou ...,6e888b3f6e


In [12]:
passages_set[passages_set["cdtnId"] == doc2]

,index,source,cdtnId,title,slug,answers.generic.text,tokens,text,cdtnId-fiches
4751,23,contributions,1100b865de,"Si le salarié est malade pendant ses congés, q...",si-le-salarie-est-malade-pendant-ses-conges-qu...,NaN,261,le salarié est malade avant son départ en cong...,NaN
4752,24,contributions,1100b865de,"Si le salarié est malade pendant ses congés, q...",si-le-salarie-est-malade-pendant-ses-conges-qu...,NaN,261,d'un arrêt maladie. le salarié perçoit une ind...,NaN


In [277]:
doc_passages = passages_set[passages_set["cdtnId"] == search_id]
indices = doc_passages.index
tensor_docs[indices]
retrieval_scores = (tensor_docs[indices] @ tensor_docs.T) * 100
retrieval_scores.size()
topk = torch.topk(retrieval_scores, 100)


In [274]:
topk.indices.size()

torch.Size([3, 100])

In [275]:
top_indices = topk.indices.tolist()

In [282]:
passages_set.loc[top_indices[0][:10]].text.iloc[doc_passages.shape[0]]

"l'employeur apporte la justification de la perturbation du fonctionnement de l'entreprise . il justifie également de la nécessité de remplacer définitivement le salarié absent par un cdi . indemnités si le salarié en remplit les conditions, il a droit aux indemnités suivantes : indemnité légale ou conventionnelle de licenciement indemnité compensatrice de congés payés indemnité compensatrice de préavis à noter des dispositions conventionnelles peuvent prévoir des conditions d'indemnisation plus favorables. motif disciplinaire conditions le salarié peut être licencié s'il a commis une faute . elle peut être la conséquence de l'une des situations suivantes : non-respect par le salarié malade de ses obligations (par exemple, l'envoi tardif d'un arrêt de travail malgré les mises en demeure de justifier de l'absence) faute commise avant l'arrêt maladie procédure disciplinaire engagée avant l'arrêt maladie procédure l'employeur doit respecter la procédure de licenciement pour motif personne

In [281]:
doc_passages.text.iloc[0]

"la clause de non-concurrence est une clause insérée dans le contrat de travail. elle vise à limiter la liberté d'un salarié d'exercer, après la rupture de son contrat, des fonctions équivalentes chez un concurrent ou à son propre compte. pour être valable, la clause doit respecter certains critères. critères de validité la clause de non-concurrence n'est pas définie par la loi , mais a été précisée par des décisions de justice, c'est-à-dire par l a jurisprudence . pour qu'elle soit applicable, la clause de non-concurrence doit répondre à certains critères cumulatifs définis qui conditionnent sa validité. la clause doit être écrite dans le contrat de travail (ou prévue dans la convention collective ). la clause est applicable uniquement si elle veille à protéger les intérêts de l'entreprise (quand le salarié est en contact direct avec la clientèle par exemple). elle ne doit pas empêcher le salarié de trouver un emploi ailleurs. la clause de non-concurrence s'applique : dans le temps (s

3

,index,source,cdtnId,title,slug,answers.generic.text,tokens,text,cdtnId-fiches
4745,17,contributions,10d3a4a5dd,Quelle est la durée de préavis en cas de licen...,quelle-est-la-duree-de-preavis-en-cas-de-licen...,NaN,1,nan,NaN


,index,source,cdtnId,title,slug,answers.generic.text,tokens,text,cdtnId-fiches
0,0,fiches_ministere_travail,100c1454c5,La prise en charge des frais de transport par ...,la-prise-en-charge-des-frais-de-transport-par-...,-,61,à savoir dans le contexte actuel de forte infl...,100c1454c5-01
1,1,fiches_ministere_travail,100c1454c5,La prise en charge des frais de transport par ...,la-prise-en-charge-des-frais-de-transport-par-...,-,1406,l'employeur a l'obligation de prendre en charg...,100c1454c5-11
2,2,fiches_ministere_travail,100c1454c5,La prise en charge des frais de transport par ...,la-prise-en-charge-des-frais-de-transport-par-...,-,1406,code du travail relatifs à la prise en charge ...,100c1454c5-11
3,3,fiches_ministere_travail,100c1454c5,La prise en charge des frais de transport par ...,la-prise-en-charge-des-frais-de-transport-par-...,-,1406,"et militaires de l'état, des collectivités ter...",100c1454c5-11
4,4,fiches_ministere_travail,100c1454c5,La prise en charge des frais de transport par ...,la-prise-en-charge-des-frais-de-transport-par-...,-,1406,"à leur lieu de travail ; acheter, pour payer c...",100c1454c5-11
...,...,...,...,...,...,...,...,...,...
4790,62,contributions,1a6a731104,Quelle est la durée maximale de la période d'e...,quelle-est-la-duree-maximale-de-la-periode-des...,NaN,914,le code du travail prévoit la durée maximale d...,NaN
4791,63,contributions,1a6a731104,Quelle est la durée maximale de la période d'e...,quelle-est-la-duree-maximale-de-la-periode-des...,NaN,914,un accord d'entreprise en prévoit une. la duré...,NaN
4792,64,contributions,1a6a731104,Quelle est la durée maximale de la période d'e...,quelle-est-la-duree-maximale-de-la-periode-des...,NaN,914,l'entreprise au cours des 3 mois précédant l'e...,NaN
4793,65,contributions,1a6a731104,Quelle est la durée maximale de la période d'e...,quelle-est-la-duree-maximale-de-la-periode-des...,NaN,914,cette embauche est effectuée dans un emploi co...,NaN


In [285]:
def find_similar(cdtnId, n):
    k = 20

    doc_passages = passages_set[passages_set["cdtnId"] == search_id]
    indices = doc_passages.index
    tensor_docs[indices]
    retrieval_scores = (tensor_docs[indices] @ tensor_docs.T) * 100
    retrieval_scores.size()
    topk = torch.topk(retrieval_scores, k)
    
    matches = topk.values.reshape(-1,).tolist()
    matching_indices = topk.indices.reshape(-1,).tolist()

    rows = passages_set.loc[matching_indices].copy()
    rows["similarity"] = matches
    results = rows[rows["cdtnId"] != cdtnId]

    return results

find_similar(search_id, 10)

,index,source,cdtnId,title,slug,answers.generic.text,tokens,text,cdtnId-fiches,similarity
1037,1037,fiches_ministere_travail,5d4a0ee541,"La démission : le refus, par le salarié, d’acc...",la-demission#Le-refus-par-le-salarie-d-accepte...,-,53,le refus du salarié d'accepter un changement d...,5d4a0ee541-5361,91.448868
2501,2501,fiches_ministere_travail,f23c2cb0a5,La modification du contrat de travail,la-modification-du-contrat-de-travail,-,22,à savoir même si la modification du contrat de...,f23c2cb0a5-13061,90.729454
977,977,fiches_ministere_travail,57e0d5b459,La procédure en cas de licenciement pour motif...,la-procedure-en-cas-de-licenciement-pour-motif...,-,35,a savoir lorsque le licenciement concerne un s...,57e0d5b459-4941,90.494431
1756,1756,fiches_ministere_travail,a587129505,La procédure en cas de licenciement individuel...,la-procedure-en-cas-de-licenciement-individuel...,-,40,a savoir ! lorsque le licenciement touche conc...,a587129505-9171,89.906990
2492,2492,fiches_ministere_travail,f1ce3dd794,La procédure de licenciement économique de 2 à...,la-procedure-de-licenciement-economique-de-2-a...,-,36,a savoir en cas de licenciement d'un salarié b...,f1ce3dd794-13031,89.798668
...,...,...,...,...,...,...,...,...,...,...
1818,1818,fiches_ministere_travail,aa36eed214,La priorité de réembauche : quelles sont les s...,la-priorite-de-reembauche#Quelles-sont-les-san...,-,377,l'employeur peut être condamné : en cas de non...,aa36eed214-9531,92.121735
2264,2264,fiches_ministere_travail,d8c49c59dd,L’indemnité légale de licenciement : comment c...,lindemnite-legale-de-licenciement#Comment-calc...,-,246,"salarié pendant cette période, n'est prise en ...",d8c49c59dd-11951,92.110054
1259,1259,fiches_ministere_travail,72d11f718c,Intéressement à la reprise d’activité des béné...,interessement-a-la-reprise-dactivite-des-benef...,-,196,dès lors que le bénéficiaire de l'allocation d...,72d11f718c-6641,92.098953
2148,2148,fiches_ministere_travail,cebfc380f2,PSE : quelles sont les entreprises concernées ...,plan-de-sauvegarde-de-lemploi-pse#Quelles-sont...,-,249,toute entreprise d'au moins 50 salariés qui pr...,cebfc380f2-11251,92.078415


In [286]:
indices = passages_set[passages_set["cdtnId"] == doc1].index
indices
indices.to_list()

[3568, 3569, 3570, 3571]

In [307]:

search_id = '6e888b3f6e'


print(passages_set[passages_set["cdtnId"] == search_id].iloc[0][["source", "slug"]].to_list())
print(passages_set[passages_set["cdtnId"] == search_id].shape[0])
#print(passages_set[passages_set["cdtnId"] == search_id]["text"].to_list())

res = find_similar(search_id, 10)
gb = res.groupby(["cdtnId", "slug", "source"])

#print( gb["similarity"].max())

scores = gb["similarity"].max() * (1+gb["similarity"].count() / 10)
scores.sort_values(ascending=False)[:10]

#grouped.mean(["similarity"]).sort_values(by=["similarity"], ascending=False)[:20]
#grouped.size()

# counts = gb.size().to_frame(name='counts')
# (counts
#     .join(gb.agg({'col3': 'mean'}).rename(columns={'col3': 'col3_mean'}))
#     .join(gb.agg({'col4': 'median'}).rename(columns={'col4': 'col4_median'}))
#     .join(gb.agg({'col4': 'min'}).rename(columns={'col4': 'col4_min'}))
#     .reset_index())
   



#.sort_values(by=["similarity"], ascending=False)[:20]

['fiches_service_public', 'arret-maladie-et-conges-payes-annuels']
4


cdtnId      slug                                                                                       source                  
fcf7934672  les-conges-payes#Quelles-sont-les-consequences-de-la-maladie-durant-les-conges-payes-nbsp  fiches_ministere_travail    211.355157
1100b865de  si-le-salarie-est-malade-pendant-ses-conges-quelles-en-sont-les-consequences               contributions               173.354068
36c2fc8868  arret-maladie-pendant-le-preavis-quelles-consequences                                      fiches_service_public       170.879604
4c949978ea  conge-ou-temps-partiel-pour-creation-ou-reprise-dentreprise                                fiches_service_public       161.863470
fcf7934672  les-conges-payes#Qui-a-droit-aux-conges-payes                                              fiches_ministere_travail    152.561145
a85dc7dbd0  un-employeur-peut-il-refuser-des-conges-demandes-par-le-salarie                            fiches_service_public       151.724451
dd59ddcb89  comment-

In [241]:
def lookup(slug):
    slug_search = passages_set[passages_set["slug"] == slug]
    dd = slug_search["cdtnId"].drop_duplicates()
    n = dd.shape[0]
    if n == 1 :
        return dd.iloc[0]
    elif n == 0 :
        raise Exception("not found")
    else :
        raise Exception("ambiguous slug search")

In [254]:
lookup("quest-ce-quune-clause-de-non-concurrence")

'c5358a2fe7'

In [78]:
passages_set[passages_set["cdtnId"] == "c6c206dc8b"]

,index,source,cdtnId,title,slug,answers.generic.text,tokens,text,cdtnId-fiches
4274,4274,fiches_service_public,c6c206dc8b,Le contrat de travail est-il obligatoirement é...,le-contrat-de-travail-est-il-obligatoirement-e...,-,249,les contrats de travail suivants doivent oblig...,c6c206dc8b
4275,4275,fiches_service_public,c6c206dc8b,Le contrat de travail est-il obligatoirement é...,le-contrat-de-travail-est-il-obligatoirement-e...,-,249,le salarié est présumé être en cdi à temps ple...,c6c206dc8b
